# General architecture

![](arch.png)

The system includes
▪ A network of wireless modules that will form the target development platform.
These nodes are directly connected to the physical sensors serving
acquisition of agri-environmental data measurements (humidity, luminosity, temperature).
The modules used are STM32 B_L072Z_LRWAN1 Discovery Kit boards. These modules are connected to external sensors installed at the level of the physical environment (ground,air) and are equipped with a LoRa module CMWX1ZZABZ-091.
     ▪ A Border Router (6LBR) or Gateway which is a Raspberry Pi3 + STM32 BOARD B_L072Z  which allows to relay measurements to the Internet.

# STM 32 B-L072Z-LRWAN1
## Description
The B-L072Z-LRWAN1 LoRa®/Sigfox™ Discovery kit is a development tool to learn and develop solutions based on LoRa®, Sigfox™, and FSK/OOK technologies. This Discovery kit features an all-in-one open module CMWX1ZZABZ-091 (by Murata). The module is powered by an STM32L072CZ and an SX1276 transceiver. The transceiver features the LoRa® long-range modem, providing ultra-long-range spread spectrum communication and high interference immunity, minimizing current consumption. Since CMWX1ZZABZ-091 is an open module, user has access to all STM32L072 peripherals such as ADC, 16-bit timer, LP-UART, I2C, SPI and USB 2.0 FS (supporting BCD and LPM).





![](stm.jpg)

## Features
The B-L072Z-LRWAN1 Discovery kit offers the following features:
• A CMWX1ZZABZ-091 LoRa® module from Murata embedding:
– STM32L072CZY6 microcontroller featuring 192 Kbytes of Flash memory, 20
Kbytes of RAM, 20 Kbytes of EEPROM
– SX1276 low-power transceiver which features a LoRa® long-range modem
• ARM® mbed™ (see http://mbed.org)
• On-board ST-LINK/V2 with selection mode switch to use the kit as a standalone
ST-LINK/V2 (with SWD connector for programming and debugging)
• Board powered through the USB bus or from an external 3V or 5V supply voltage or
battery
• Six LEDs:
– LD7 (red/green) for USB communication
– LD2 (red) for 3.3V power-on
– Four user LEDs: LD1 (Green), LD2 (red), LD3 (blue) and LD4 (red)
• Two push-buttons (user and reset)
• USB with Micro-B connector for the CMWX1ZZABZ-091 modue
• Arduino™ connector to plug Arduino/Mbed compatible expansion board.
• Extension header for a quick connection to the prototyping board and an easy probing

# Pin mapping
## Sender node
![](cap.png)


For our external sensors(humidity and luminosity) we used the pin PA0 to read the raw output of the ADC channel 0 for the humidity as for the luminosity we used the pin PA4 to read the raw output of the ADC channel 4.

# Data acquisition

Our sensors are all analog sensors which is why we will be using the ADC of our card to read the data from the sensors. The temperature sensor is internal linked to the temperature channel of the ADC while the humidity and luminosity 
sensors are external thus linked to the channels 0 and 4 respectively .

## Temperature

We used as stated above the internal temperature sensor of our card.

### Code

#### acquire data

In [ ]:
measuredLevel0 = HW_AdcReadChannel( ADC_CHANNEL_TEMPSENSOR ) : read the temperature value from the ADC

#### Calibration 

In [ ]:
 	#define COMPUTE_TEMPERATURE(TS_ADC_DATA, VDDA_APPLI)                           \
((((( ((int32_t)(((TS_ADC_DATA) * VDDA_APPLI) / VDDA_TEMP_CAL)                  \
 - (int32_t) *TEMP30_CAL_ADDR)                                          \
    ) * (int32_t)(110 - 30)                                                   \
      )) / (int32_t)(*TEMP110_CAL_ADDR - *TEMP30_CAL_ADDR)                        \
        ) + (30)                                                                      \

          )
    temperatureDegreeC1 = COMPUTE_TEMPERATURE( measuredLevel0, batteryLevelmV1);

In [ ]:
TS_CAL2 is temperature sensor calibration value acquired at 110°C found at the mmemory address 0x1FF8 007A 
TS_CAL1 is temperature sensor calibration value acquired at 30°C  found at the mmemory address 0x1FF8 007E 
TS_ADC_DATA is the actual temperature sensor output value converted by ADC 

## Humidity

Sensor We used a basic low-cost soil moisture sensor. This sensor has an analogue
output that can be connected to the PIN PA0 with a voltage of 3.3V .
The soil moisture sensor consists of two probe’s pads which are used to measure the
Volumetric content of water. The two probe’s pads allow the current to pass through
the soil and then it gets the resistance value to measure the moisture value.
• Wet soil : with higher volume of water, conductivity increases and the resistance
will decrease. Therefore the output voltage will be less.
• Dry soil : poor conductivity and so the resistance increases and the output
voltage will be high.
Humidity reading The measured soil conductivity and the humidity are inversely
related. To get the correct humidity reading we note the extreme values of the
measured data in two fields : dry and wet soil. We get the following :
Max Raw value (Dry soil) 4095
Min Raw Value (Wet soil) 740
The measured values present the raw values of humidity which is the output of the
ADC register (12 bits). Given the max and min values, we established the following
formula to retrieve the humidity in percentage from a measured HumidityRAW Value.

![](a.png)


### Code

#### Data acquisiton 

In [ ]:
measuredLevel_humidity= HW_AdcReadChannel(ADC_CHANNEL_0); reads the raw value of the output of our ADC

#### Calibration

In [ ]:
hum= ((float)((((Buffer[3] << 8) | Buffer[2])-(float)740)/((float)4095-(float)740))*100) ;

## Luminosity

In [ ]:
The sensor specifications are listed below.
Light Level Min 1 lx
Light Level Max (3.3V) 46200lx
Light Level Max (5V) 70000 lx
Since we already used the 3.3V VCC pin for the humidity sensor we will use the 5V pin for the luminosity sensor
Given the maximum and the minimum value of the luminosity raw output we established this formula

![](b.png)

### Code 

#### Data acquisition

In [ ]:
measuredLevel_luminosity = HW_AdcReadChannel(ADC_CHANNEL_4 );

#### Calibration

In [ ]:
lum=(uint16_t)((((Buffer[5] << 8) | Buffer[4])*(float)70000)/((float)2586-(float)321)*((float)5/(float)3));

# Communication between nodes 

## Sender 

To communicate between diffrent nodes through the  LRWAN1 module we used the functions included in the library radio.h of the stm32.

### Code 

First we defined our nodeid which will identify each node

In [ ]:
uint8_t  nodeid=1;

First we defined a Buffer that we will fill and send to the reciever node :

In [ ]:
uint8_t Buffer[6];

then we initialised the node as a sender node:

In [ ]:
RadioEvents.TxDone = OnTxDone;

where OnTxDone is defined as such :

In [ ]:

void OnTxDone( void )
{
  Radio.Sleep( );
  State = TX;
  PRINTF("OnTxDone\n\r");
}

After that we filled the buffer in this order : nodeid,temperature,humidite,luminosite
We divided the humidity and luminosity on two sperate Buffer Boxes because our lrwan1 module sends in bytes.

In [ ]:
    Buffer[0]= (uint8_t)nodeid;
    Buffer[1]=(uint8_t)temperatureDegreeC1; 
    Buffer[2]=(uint8_t)( measuredLevel_humidity & 0xff); 
    Buffer[3] =( measuredLevel_humidity>>8); 
    Buffer[4] =( measuredLevel_luminosity & 0xff); 
    Buffer[5] =( measuredLevel_luminosity>>8);

Finally the packet transmission is done by the function radio.send which takes as parameters the buffer and its size

In [ ]:
Radio.Send(Buffer, sizeof(Buffer));

## Reciever 

### Code

we initialised our node as a reciever node

In [ ]:
RadioEvents.RxDone = receiver;

with reciever defined as :

In [ ]:
void receiver( uint8_t *payload, uint16_t size, int16_t rssi, int8_t snr )
{
  Radio.Sleep( );
  BufferSize = size;
  memcpy(Buffer, payload, BufferSize );
}

then we sent the calibrated the values recieved as shown in the sensor description part
After that we created a new buffer to be sent to our gateway the Rasberry Pi card through a USB connection

In [ ]:
char  buffer_usart[100]; 

and we filled it with the data calibrated using sprintf

In [ ]:
sprintf(buffer_usart,"%d:%d:%d:%d\n",(uint8_t) nodeid,(uint8_t) temp,(uint8_t) hum,(uint8_t) lum);

Finally we sent the new buffer to the Rasberry Pi through a USB connection

In [ ]:
vcom_Trace((uint8_t*)buffer_usart, sizeof(buffer_usart) );